In [ ]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,093 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,073 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://cloud.r-project.org/bin

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-25 00:30:57--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.58MB/s    in 0.6s    

2022-08-25 00:30:58 (1.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NFL-Prediction").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://nflpredictionsdataviz.s3.amazonaws.com/spreadspoke_scores.csv"
spark.sparkContext.addFile(url)
NFL_Guru_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("spreadspoke_scores.csv"), sep=",", header=True, inferSchema=True)
NFL_Guru_df.show()

+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|          team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|     9/2/1966|           1966|            1|           false|     Miami Dolphins|        14|        23|     Oakland Raiders|            null|           null

In [ ]:
NFL_Guru_df.printSchema()

root
 |-- schedule_date: string (nullable = true)
 |-- schedule_season: integer (nullable = true)
 |-- schedule_week: string (nullable = true)
 |-- schedule_playoff: boolean (nullable = true)
 |-- team_home: string (nullable = true)
 |-- score_home: integer (nullable = true)
 |-- score_away: integer (nullable = true)
 |-- team_away: string (nullable = true)
 |-- team_favorite_id: string (nullable = true)
 |-- spread_favorite: double (nullable = true)
 |-- over_under_line: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- stadium_neutral: boolean (nullable = true)
 |-- weather_temperature: integer (nullable = true)
 |-- weather_wind_mph: integer (nullable = true)
 |-- weather_humidity: integer (nullable = true)
 |-- weather_detail: string (nullable = true)

